#<font color=blue>**YOLO를 통한 버클리 데이터셋 학습**</font>

##<font color=green>**구글 드라이브 연결**</font>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##<font color=green>**darknet yolo 폴더로 이동**
2_04_YOLO 객체 인식 수업에서 이미 설치를 진행함

In [2]:
cd drive/MyDrive/darknet/

/content/drive/MyDrive/darknet


##<font color=green>**yolov3 convolution layer 모델 설치**</font>

In [3]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-01-17 00:16:18--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  9.86MB/s    in 16s     

2021-01-17 00:16:35 (9.66 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



##<font color=green>**버클리 데이터 업로드 진행**</font>

17GJj5XoLYMw_DxvH486BV8c08ycExRWn

In [7]:
%cd data/
!gdown --id 1_MLsDHKmoPSLBe3WuN1jrLWAvQoaOvQ2 --output BDD_data.zip

Downloading...
From: https://drive.google.com/uc?id=1_MLsDHKmoPSLBe3WuN1jrLWAvQoaOvQ2
To: /content/drive/My Drive/darknet/BDD_data.zip
346MB [00:04, 81.7MB/s]


In [10]:
!unzip BDD_data.zip

Archive:  BDD_data.zip
replace bdd.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!./darknet detector train data/bdd.data data/yolov3-bdd100k.cfg darknet53.conv.74 -dont_show